<a href="https://colab.research.google.com/github/Dromarion/Dog-Breed-Classification-Model/blob/experiments-cleanup/dog_vision_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dog Breed Classification Model Experiments
This notebook will be used for testing various experiments to create a multi-class image classifier using Tensorflow.

## Objectives
If I take a photo of a dog, I want to know what breed the dog is.

## Data
The data used for this project will be from Kaggle's dog breed identification dataset. This should be stored in the google drive inside 

/<wbr>content/drive/MyDrive/Data/Dog Vision/

https://www.kaggle.com/c/dog-breed-identification/data

## Features
* There are 120 unique breeds of dogs, some notable breeds seem to be missing, for example Huskies.
* There are 10,000+ images in each of the training and test datasets. The training images have labels while the test datasets do not.

In [12]:
# Import Libraries
import os
import datetime
from IPython.display import Image
# Data Science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub

In [20]:
# Create variables for training labels and filepaths
labels_csv = pd.read_csv("/content/drive/MyDrive/Data/Dog Vision/labels.csv")
filepath = ["/content/drive/MyDrive/Data/Dog Vision/train/" + fname + ".jpg" for fname in labels_csv["id"]]
# Verify if filepaths match actual number of files in directory
len(filepath) == len(os.listdir("/content/drive/MyDrive/Data/Dog Vision/train/"))

True

In [28]:
# Create array of all breed labels in csv
labels = labels_csv["breed"].to_numpy()
# Create array of the 120 unique breeds
unique_breeds = np.unique(label)
# Turn each label into a series of booleans of all 120 unique breeds
boolean_label = [label == unique_breeds for label in labels]